## Задание №6

Большие языковые модели (БЯМ) в последние годы стали быстро расти. Точная настройка таких больших моделей под индивидуальные задачи обходится непомерно дорого. Для ускорения обучения можно применять низкоранговые разложения.

Давайте исследуем произвольный промежуточный слой обученной БЯМ, который можно описать уравнением:

$Y = W_{n \times m} \cdot X$,

где $W_{n \times m}$ — матрица обученных параметров размера $n \times m$, $X$ — входные активации, $Y$ — выходные активации, $\cdot$ — операция матричного умножения.

Предположим, что изменение весов во время адаптации модели (дообучении на целевую задачу) будет иметь низкий "внутренний ранг" $r$. Если это так, то мы можем имитировать эффект полноценной тренировки, обучая вместо параметров $W_{n \times m}$ две небольшие матрицы низкого ранга $A$ и $C$. В результате, слой сети будет описываться уравнением:

$Y = W_{n \times m}^{frozen} \cdot X + A_{n \times r} \cdot C_{r \times m} \cdot X$,

где $W_{n \times m}^{frozen}$ — зафиксированная матрица обученных параметров, $A_{n \times r}$ и $C_{r \times m}$ — обучаемые параметры.

На примере части графа вычисления БЯМ для $n = 3$ и $m = 10$ определите, скелетные разложения каких рангов $r_i$ необходимо найти для каждого слоя, чтобы уменьшить количество обучаемых параметров в сумме в 3 раза. Слой `MatMul` с припиской $B_{⟨x, y⟩}$ на изображении обозначает матричное умножение на матрицу весов размерности $(x, y)$. Слой `Split` разделяет вектор входов на несколько частей, `Transpose` транспонирует матрицу входов.

**Формат вывода:** Четыре целых числа $r_i > 1$, записанные через запятую без пробелов, обозначающие ранги найденных матриц для каждого из слоев сети. Очередность слоев задается нумерацией по изображению от одного до четырех и последовательно по приведенному коду (см. функцию `forward`).

**Примечания:** Для лучшего понимания предложенного графа вычислений приводим программный код на языке Python, написанный с применением библиотеки torch:

```python
class CustomGraph(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qkv = torch.nn.Linear(10, 30, bias=False)
        self.q_proj = torch.nn.Linear(10, 5, bias=False)
        self.k_proj = torch.nn.Linear(10, 5, bias=False)
        self.v_proj = torch.nn.Linear(10, 20, bias=False)

    def forward(self, x):
        q, k ,v = torch.split(self.qkv(x), 10, dim=1)
        qk_proj = torch.matmul(self.q_proj(q), self.k_proj(k).T)
        return torch.matmul(qk_proj, self.v_proj(v))
```

In [11]:
import torch
from torchviz import make_dot

class CustomGraph(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.qkv = torch.nn.Linear(10, 30, bias=False)
        self.q_proj = torch.nn.Linear(10, 5, bias=False)
        self.k_proj = torch.nn.Linear(10, 5, bias=False)
        self.v_proj = torch.nn.Linear(10, 20, bias=False)

    def forward(self, x):
        q, k ,v = torch.split(self.qkv(x), 10, dim=1)
        qk_proj = torch.matmul(self.q_proj(q), self.k_proj(k).T)
        return torch.matmul(qk_proj, self.v_proj(v))

model = CustomGraph()
x = torch.randn(1, 10)
output = model(x)
make_dot(output).render("graph", format="png")

'graph.png'

![graph](./graph.png)


In [12]:
matrices = [(10, 30), (10, 5), (10, 5), (10, 20)]
ranks = [2, 2, 2, 2]

total_params = sum(n * m for n, m in matrices)
compressed = sum(ranks[i] * n + ranks[i] * m for i, (n, m) in enumerate(matrices))
total_params, compressed

(600, 200)

In [13]:
print('Ранги:')
print(', '.join(map(str, ranks)))
print('Количество параметров:')
print(compressed)

Ранги:
2, 2, 2, 2
Количество параметров:
200
